First, some setup code, so we can run this on Colab. No need to run this on a local machine.

In [2]:
!git clone https://github.com/smeenehan/TensorFlowProjects.git

fatal: destination path 'TensorFlowProjects' already exists and is not an empty directory.
/content/TensorFlowProjects/CIFAR


In [3]:
%cd TensorFlowProjects/CIFAR

/content/TensorFlowProjects/CIFAR


In [1]:
from model import ResNet, compute_accuracy
import tensorflow as tf
from train import ModelTrainer
import utils
%matplotlib inline

c:\users\smmeenehan\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
eager_mode = True
if eager_mode:
    tf.enable_eager_execution()

In [3]:
tf.reset_default_graph()

In [2]:
device, data_format = utils.device_and_data_format()
train_data, val_data, test_data, x_test_raw = utils.get_CIFAR10_data(data_format)

In [3]:
model = ResNet(data_format)
loss = tf.losses.sparse_softmax_cross_entropy
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)

In [4]:
trainer = ModelTrainer(model, compute_accuracy, loss, optimizer, val_data, val_data, val_data)

In [5]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    trainer.train(num_epochs=5)


Initial validation loss, accuracy: 62.001305, 0.101367

Train time for epoch #1: 20.285028

Validation loss, accuracy for epoch #1: 75.869097, 0.111328

Train time for epoch #2: 17.980798

Validation loss, accuracy for epoch #2: 22.747970, 0.110156

Train time for epoch #3: 18.966897

Validation loss, accuracy for epoch #3: 7.807427, 0.130469

Train time for epoch #4: 18.890889

Validation loss, accuracy for epoch #4: 2.614957, 0.212695

Train time for epoch #5: 16.701670

Validation loss, accuracy for epoch #5: 2.350058, 0.243555
